<a href="https://colab.research.google.com/github/Segtanof/pyfin/blob/main/08_Regressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regressions
Useful links:
- https://aeturrell.github.io/coding-for-economists/econmt-regression.html
<!-- - https://aeturrell.github.io/python4DS/quarto.html -->

We need to install some packages.
We use `pyfixest` for regressions and `pingouin` for ttests / correlation analyses.

In [1]:
pip install pyfixest

  Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 12.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ------------------------------------- -- 2.9/3.1 MB 13.9 MB/s eta 0:00:01
   ---------------------------------------- 3.1/3.1 MB 14.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/35.2 MB ? eta -:--:--
   -- ------------------------------------- 2.4/35.2 MB 12.3 MB/s eta 0:00:03
   ------ --------------------------------- 5.8/35.2 MB 14.1 MB/s eta 0:00:03
   --------- ------------------------------ 8.7/35.2 MB 14.1 MB/s eta 0:00:02
   ------------ --------------------------- 11.3/35.2 MB 13.8 MB/s eta 0:00:02
   ---------------- ----------------------- 14.7/35.2 MB 14.2 MB/s eta 0:00:02
   -------------------- ------------------- 18.4/35.2 MB 14.7 MB/s eta 0:00:02
   ---------------

In [2]:
# General packages
import pandas as pd
import numpy as np
import seaborn as sns

# Regression-specific packages
import pyfixest as pf
import pingouin as pg

In [3]:
# We load a dataset of NYC taxi trips
# Source: https://github.com/mwaskom/seaborn-data

# Note that we drop rows that have any missing values
data = sns.load_dataset("taxis").dropna(how='any')

In [4]:
data.sample(3)

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
1373,2019-03-08 23:58:46,2019-03-09 00:05:18,1,1.20,6.5,2.55,0.0,12.85,yellow,credit card,Greenwich Village North,Hudson Sq,Manhattan,Manhattan
1334,2019-03-03 00:40:55,2019-03-03 00:43:51,1,0.66,4.5,1.66,0.0,9.96,yellow,credit card,Hudson Sq,Little Italy/NoLiTa,Manhattan,Manhattan
6312,2019-03-07 09:48:29,2019-03-07 10:02:20,1,2.02,11.0,1.77,0.0,13.57,green,credit card,Upper East Side North,Central Harlem,Manhattan,Manhattan


## T-test
A common hypothesis test is the t-test.
We can check whether the mean of an array differs from a value as follows.

Let's check if `tolls` are significantly different from `0`.

In [5]:
value_to_test_against = 0
pg.ttest(data['tolls'], value_to_test_against)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,18.308157,6340,two-sided,5.280784e-73,"[0.28, 0.35]",0.229914,1.11e+69,1.0


Interpretation: We do not find support for the null-hypothesis of `tolls` being equal to `0` and thus reject it. We can see this from the t statistic being larger than the critical value, or, equivalently, the `p-val` being lower than common significance thresholds.

If we arbitrarily limit the number of observations by taking a random `sample`, we can see more uncertainty.

Because it's a random sample, you won't see the exact same values.

In [12]:
sample = data['tolls'].sample(50)

In [7]:
pg.ttest(sample, value_to_test_against)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.768519,49,two-sided,0.083199,"[-0.05, 0.74]",0.250106,0.652,0.410667


**Quick exercise**

You can see that we are running a two-sided t-test. Since tolls cannot be negative a one-sided t-test might be more appropriate.

Run a one-sided t-test.

In [13]:
value_to_test_against = 0
pg.ttest(sample, value_to_test_against, alternative="greater")

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.428869,49,greater,0.079693,"[-0.04, inf]",0.202073,0.797,0.406819


A more realistic test you might want to run is comparing the different trip distances based on the color of the cab.

![Taxis](https://newyorkmonamour.fr/wp-content/uploads/2016/10/boro-cabs-vs-yellow-cabs.jpg)

We can check how many observations are in each color by using the `value_counts` method:

In [14]:
data.value_counts(subset="color")

color
yellow    5373
green      968
Name: count, dtype: int64

We can check the means in each group:

In [18]:
data.groupby("color")["distance"].mean()

color
green     3.412665
yellow    2.922948
Name: distance, dtype: float64

It looks like green taxis travel about a half-mile further than yellow taxis. But is this difference statistically significant or just a random finding?

In [21]:
yellow_distance = data.query("color == 'yellow'")['distance']
green_distance = data.query("color == 'green'")['distance']
green_distance

5451     2.29
5452     0.80
5453     1.51
5454     0.45
5455     0.61
        ...  
6428     0.75
6429    18.74
6430     4.14
6431     1.12
6432     3.85
Name: distance, Length: 968, dtype: float64

In [23]:
# This time we are using some additional keyword arguments: confidence
# Check the documentation to see what that does: https://pingouin-stats.org/build/html/generated/pingouin.ttest.html#pingouin.ttest

pg.ttest(yellow_distance, green_distance, confidence=0.99)

,T,dof,alternative,p-val,CI99%,cohen-d,BF10,power
T-test,-3.546353,1274.846737,two-sided,0.000405,"[-0.8459478061155647, -0.13348662499314595]",0.13179,20.428,0.965151


The result shows that the mean of the yellow taxis is statistically significantly less than the mean trip distance of green taxis.

**Quick exercise**

- How many unique pickup zones are in the data set?

- Check whether the `total` price paid differs between the `pickup_zone` "Yorkville East" and "Yorkville West" and whether the difference is significant at the 10% level. Show the appropriate confidence interval.

In [25]:
data["pickup_zone"].nunique()

194

In [26]:
data.head()

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan
3,2019-03-10 01:23:59,2019-03-10 01:49:51,1,7.70,27.0,6.15,0.0,36.95,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan
4,2019-03-30 13:27:42,2019-03-30 13:37:14,3,2.16,9.0,1.10,0.0,13.40,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan


In [35]:
yet = data.query("(pickup_zone == 'Yorkville East')")["total"]
yet.mean()

15.59285714285714

In [36]:
ywt = data.query("(pickup_zone == 'Yorkville West')")["total"]
ywt.mean()

14.948415841584158

In [50]:
pol = ["Yorkville West", "Yorkville East"]
data.query("pickup_zone in @pol").groupby("pickup_zone")["total"].mean()

pickup_zone
Yorkville East    15.592857
Yorkville West    14.948416
Name: total, dtype: float64

In [42]:
pg.ttest(yet, ywt, confidence=.9)

,T,dof,alternative,p-val,CI90%,cohen-d,BF10,power
T-test,0.624414,149.153958,two-sided,0.533309,"[-1.0637778932951096, 2.3526604958410737]",0.096997,0.201,0.095092


## Linear regression

### Basics

Let's try to see if we can figure out the pricing structure.
In order to do that, t-tests are not sufficient, because we need to consider more than a single variable.

Typically, taxi fares start at a base fare, a per-mile rate, and a time component. In NYC, there are [other components](https://www.nyc.gov/site/tlc/passengers/taxi-fare.page) as well.

From those three components, we can easily observe the base fare (intercept) and the per-mile rate (distance).

In order to specify the regression equation, we use the `~` (tilda) operator to separate the left- and right-hand sides of the regression equation.

We then add the variable names (column names) from our data. **Avoid spaces in the column names!**

To regress `y` on `x`, use `y ~ x`, which covers the regression equation $y = a + \beta x + \epsilon$. An intercept is added automatically. If you do not want to add an intercept, set the keyword argument `drop_intercept=True`.

In [ ]:
pf.feols("fare ~ distance", data=data).summary()

It looks like the base fare is \$4.7 and a charge of \$2.73 is added per mile.
Our estimates are statistically significant and we can explain about 90% of the variance in the data.

**Quick exercise**

- Write the regression formula for the regression equation $R_t = a + \beta R_M + \epsilon$
- Make a scatterplot. `tip` against `distance`. What do you notice?
- Run a regression for $tip = a + \beta \times distance + \epsilon$
- Run a regression for $tip = \beta \times distance + \epsilon$

#### New variables
Very often, you will need to construct your own variables.





**Trip duration**

Since we know the time of pickup and dropoff, we can calculate the duration of the trip.
We first take the difference between the two columns, and then use the `.dt` (datetime) [accessor of the resulting column](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) (which is the type pandas.Series). In the next step we pull out the seconds and divide them by 60 to get minutes. Then we store it as `duration_min`. Note that we avoid spaces in the column names.

**Time-of-day surcharges**
```txt
Plus $1.00 overnight surcharge 8pm to 6am.
Plus $2.50 rush hour surcharge from 4pm to 8pm on weekdays, excluding holidays.
```

[There are some surcharges](https://www.nyc.gov/site/tlc/passengers/taxi-fare.page) depending on the time of day.


In [ ]:
pd.Timestamp("2030-06-20 09:54:32").hour

In [ ]:
def is_rushhour(date):
  if date.dayofweek <= 4:
    if date.hour >= 16 and date.hour < 20:
      return True

  # Note that we do not need to specify all the `else` statements
  return False

In [ ]:
data_with_added_variables = data.assign(

    # Weekend
    weekend = lambda df: df['pickup'].dt.dayofweek >= 5,

    # Duration in minutes
    duration_min = lambda df: (df['dropoff']-df['pickup']).dt.seconds/60,

    # Time-of-day surcharges
    overnight = lambda df: ((df['pickup'].dt.hour >= 20) | (df['pickup'].dt.hour <= 6)),
    rushhour = lambda df: df['pickup'].apply(is_rushhour),
    )

data_with_added_variables.head(2)

**Expensive areas**

[This source](https://www.roadwaymoving.com/blog/most-expensive-neighborhoods-in-nyc/) provides a list of NYC's most expensive neighbourhoods.
We want to add a variable that tells us if a trip started in one of these expensive areas.

In [ ]:
expensive_areas = {"pickup_zone":['TriBeCa/Civic Center', 'Central Park', 'Times Sq/Theatre District', 'SoHo', 'Little Italy/NoLiTa', 'West Chelsea/Hudson Yards']}
expensive_areas = pd.DataFrame(expensive_areas).assign(expensive = True)
expensive_areas

In [ ]:
# We need to merge how='left'
data_with_added_variables = data_with_added_variables.merge(expensive_areas, how='left',on='pickup_zone')

data_with_added_variables.head(2)

Due to the `how='left'` merge, we have a lot of missing values (NaN). We want to fill them. Thus, we use the `fillna()` method on that column.

In [ ]:
data_with_added_variables['expensive'] = data_with_added_variables['expensive'].fillna(False)
data_with_added_variables.head(2)

**Using the new variables in a regression**

Let's try to see if the fare can be predicted based on the `distance` and our new variables.

In [ ]:
pf.feols("fare ~ distance + duration_min + overnight + rushhour", data=data_with_added_variables).summary()

**Quick exercise**

Only some variables are significant predictors of the fare. Many are not.

Since they are called "surcharges", maybe they don't count towards the fare, but rather towards the total? Run that regression and interpret your results.

### Advanced

#### Interaction terms

We typically require slightly more advanced approaches to address our research question.

For example interaction terms.
Let's say we want to find out whether the tip amount is dependent on whether it's a weekend (people might be on a leisure trip and willing to tip more) and the ride starts in an expensive area (where passengers are wealthier). In addition, we also want to know whether riders from expensive areas tip differently during the weekend (because wealthier people might be more likely to go on a leisure trip).

Thus, we can interact `weekend` with `expensive`. We can do so by writing `weekend*expensive`, which adds both variables individually and their interaction term.

Unfortunately, tips are only recorded for credit card payments, thus, we need to filter the data.

In [ ]:
credit_card_only = data_with_added_variables.query("payment == 'credit card'")

In [ ]:
pf.feols("tip ~ fare + weekend*expensive", data=credit_card_only).summary()

# Equivalent
# pf.feols("tip ~ fare + weekend + expensive + weekend:expensive", data=credit_card_only).summary()

From our results, we can see that in general our hypothesis were true, except that we would have expected a positive coefficient on the interaction term as well.

**Quick exercise**

- Run the regression for the following equation: $fare = a + \beta_1 \times distance + \beta_2 \times weekend + \beta_3 \times weekend \times distance + \epsilon$. What are we measuring with the interaction term?
- Calculate the unconditional mean of `fare`. How large is the economic effect of weekend in percent of the unconditional mean? Is it statistically and economically significant? (Manually copy the coefficient for weekend).

#### Variable transformations

Using the logarithm and other transformations are also very commonly applied methods. To achieve this, you can of course create a new column in your data that has the transformation applied.

Some functions (numpy) can however also be applied in your regression equation, which can be useful.

In [ ]:
pf.feols("total ~ np.log(distance+1) + tip + tolls + duration_min + overnight + rushhour", data=data_with_added_variables).summary()

## Fixed effect regression

Now let's run a more advanced type of regression. Fixed effect regressions.

Typically, we have a single intercept in our regression. However, in some cases, we might want to have separate intercepts for e.g. the color of the taxis.

We can specify that by providing the pipe operator `|` after our regression specification and then listing the different variables we want to add fixed effects for.

In [ ]:
pf.feols("fare ~ distance + duration_min | color", data=data_with_added_variables).summary()
# You can see that there is no intercept listed in the regression output anymore. It is subsumed by the Fixed Effect.

**Quick exercise**

Add a FE for `pickup_zone` to the regression of the `tip` on `fare`. Use the credit card only dataset.

## Prediction

Sometimes you want to have predicted values, e.g. when forecasting expected returns.

In [ ]:
# Let's take the following model
model = pf.feols("fare ~ distance + duration_min | color", data=data_with_added_variables) # No .summary()!

In [ ]:
# Run the `predict` method of the model.
predicted_fare = model.predict(data_with_added_variables)
predicted_fare

In [ ]:
# We can then use the predictions in any way we want, e.g. to calculate the difference to the actual data.
data_with_added_variables['fare'].subtract(predicted_fare).hist(bins=100)

## Presenting the results

After running (multiple) regressions, you typically want to show them in an easy-to-understand way. We can use the `etable` function for this purpose.

In [ ]:
# First, we run several regressions.
results = []

specifications = [
    "total ~ distance + tip + tolls*duration_min + overnight + rushhour",
    "fare ~ distance + duration_min | color + C(passengers)",
]

for spec in specifications:
  result = pf.feols(spec, data=data_with_added_variables)
  results.append(result)

In [ ]:
# Second, we show the results in a nice format.
# The coef_fmt specifies that we want the t-stat in parentheses.
pf.etable(results, coef_fmt = "b (t)")

In [ ]:
# If you want to export it to Excel, you can use type='df'. There is also latex with type='tex'
pf.etable(results, coef_fmt = "b (t)", type="df")

## Exercises

a)
- Test whether passengers are different from one.
- Test whether the amount paid as fare differs between cash and credit card payments.

b) You want to study the effect of tolls on trip duration.

- Run a regression with an intercept and without an intercept.
- Plot a scatterplot that shows the relevant data

c) You're still studying the effect of tolls on trip duration.
- Run the simple regression from b with the intercept.
- Add the distance to your regression [use this spec in part d)].
- Additionally add the overnight dummy.
- Additionally add an interaction between distance and overnight.
- Additionally add a fixed effect for the taxi color.
- Collect all of these regressions into `results` and show them in a single table. Show the coefficients and the t-statistics.

d) You're still studying the effect of tolls on trip duration.
- For the regression model highlighted in part c), calculate predicted values.
- Plot predicted vs actual values in a scatterplot
  - Set the color of the points based on the pickup_borough
  - Set the marker type based on the taxi color